# GRB Binning Pipeline for `bn081207680`
This notebook automatically downloads required data from Wasabi, unzips the response file, writes `inputs.yaml`, and bins the GRB data using `cosipy`.


In [14]:
import warnings
warnings.filterwarnings("ignore")

import os
import subprocess
import yaml
from pathlib import Path
import zipfile

from cosipy.util import fetch_wasabi_file
from cosipy.response import FullDetectorResponse
from cosipy import BinnedData
print("All libraries imported.")

All libraries imported.


In [15]:
data_folder = Path("/data01/grb")
data_folder.mkdir(parents=True, exist_ok=True)
print(f"Data directory: {data_folder}")

Data directory: /data01/grb


In [16]:
wasabi = {
    "background":  "Backgrounds/Ge/Total_BG_with_SAAcomponent_3months_unbinned_data_filtered_with_SAAcut.fits.gz",
    "response":    "Responses/ResponseContinuum.o3.e100_10000.b10log.s10396905069491.m2284.filtered.nonsparse.binnedimaging.imagingresponse_nside8.area.good_chunks.h5.zip",
    "orientation": "Orientation/DC3_final_530km_3_month_with_slew_1sbins_GalacticEarth_SAA.ori",
    "source":      "Sources/GRB_bn081207680_3months_unbinned_data_filtered_with_SAAcut.fits.gz",
}

paths = {key: data_folder / Path(val).name for key, val in wasabi.items()}

## Wasabi File Dictionary

The `wasabi` dictionary specifies relative paths to required files in the COSI Wasabi S3 storage.

| Key         | Description                                                             |
|-------------|-------------------------------------------------------------------------|
| `background`| Background FITS file (SAA-filtered)                                     |
| `response`  | Zipped detector response matrix (.h5)                                   |
| `orientation`| Orientation file for satellite pointing                                |
| `source`    | GRB photon list FITS file (SAA-filtered, unbinned)                      |

These paths are relative to the internal COSI Wasabi bucket structure.




In [17]:
for key, remote in wasabi.items():
    out = paths[key]
    ready = out.with_suffix('') if out.suffix == ".gz" else out
    if not ready.exists():
        print(f"⬇️ Downloading {key} from Wasabi...")
        fetch_wasabi_file(f"COSI-SMEX/DC3/Data/{remote}", output=out)
        if out.suffix == ".gz":
            subprocess.run(["gunzip", "-f", str(out)], check=True)
        elif out.suffix == ".zip":
            print(f"📦 Unzipping {out.name}...")
            with zipfile.ZipFile(out, 'r') as zip_ref:
                zip_ref.extractall(data_folder)
            print("✅ Unzip complete")
    else:
        print(f"✅ [Ready] {key}")

✅ [Ready] background
✅ [Ready] response
✅ [Ready] orientation
✅ [Ready] source


## Step 4: Download and Extract Files

This loop downloads and extracts each required file from Wasabi into `data_folder`, only if not already present.

| Code                             | Purpose                                      |
|----------------------------------|----------------------------------------------|
| `for key, remote in wasabi.items()` | Loop through all required file types        |
| `out = paths[key]`               | Local target path                           |
| `ready = out.with_suffix('')`   | Expected filename after extraction          |
| `if not ready.exists()`         | Download only if file doesn't exist         |





In [7]:
tmin = 1836496300.0
tmax = 1836496388.9730453
dt_bin = 1.0

## Define GRB Time Interval and Binning Size

Sets the time range and bin width for analyzing the GRB event.

| Variable   | Description                                            |
|------------|--------------------------------------------------------|
| `tmin`     | GRB start time in MET seconds                          |
| `tmax`     | GRB end time in MET seconds                            |
| `dt_bin`   | Time bin width in seconds (1-second resolution)        |





In [18]:
yaml_path = data_folder / "inputs.yaml"
src_fits = paths["source"].with_suffix('')
rsp_h5 = data_folder / "ResponseContinuum.o3.e100_10000.b10log.s10396905069491.m2284.filtered.nonsparse.binnedimaging.imagingresponse_nside8.area.good_chunks.h5"

with FullDetectorResponse.open(rsp_h5) as R:
    inputs = {
        "data_file": str(src_fits),
        "ori_file": "NA",
        "unbinned_output": "fits",
        "time_bins": dt_bin,
        "energy_bins": [float(e) for e in R.axes["Em"].edges.value],
        "phi_pix_size": 6,
        "nside": 8,
        "scheme": "ring",
        "tmin": float(tmin),
        "tmax": float(tmax)
    }

with open(yaml_path, "w") as f:
    yaml.safe_dump(inputs, f, sort_keys=False)
    print(f"✅ Written: {yaml_path}")

✅ Written: /data01/grb/inputs.yaml


## Write YAML Configuration for GRB Binning

Creates an `inputs.yaml` file that defines how the GRB FITS file will be binned into Compton Data Space (CDS).

### Key Config Parameters:

| Key              | Description                                       |
|------------------|---------------------------------------------------|
| `data_file`      | Path to the unbinned GRB FITS file                |
| `ori_file`       | Set to `"NA"` (not used)                          |
| `time_bins`      | Time bin size in seconds (`dt_bin`)               |
| `energy_bins`    | Extracted from response file                      |
| `phi_pix_size`   | Angular bin size for scatter angle Φ              |
| `nside`          | HEALPix resolution (e.g., 8 = 768 pixels)         |
| `scheme`         | HEALPix ordering scheme (`"ring"`)               |
| `tmin`, `tmax`   | GRB observation time window                       |

The generated YAML will be used in the binning step with `BinnedData`.


In [9]:
analysis = BinnedData(yaml_path)

analysis.get_binned_data(
    unbinned_data=str(src_fits),
    output_name="GRB_bn081207680_binned_O3",
    psichi_binning="local"
)
print("✅ Binning complete. Output written: GRB_bn081207680_binned_O3.hdf5")

✅ Binning complete. Output written: GRB_bn081207680_binned_O3.hdf5


## Binning GRB Data into Compton Data Space (CDS)

This step bins the unbinned GRB photon data into a 5D histogram for CDS analysis using `cosipy`.

### Key Steps:
| Code                          | Purpose                                        |
|-------------------------------|------------------------------------------------|
| `BinnedData(yaml_path)`       | Loads binning config from `inputs.yaml`        |
| `get_binned_data(...)`        | Executes binning and saves output to HDF5      |

### Main Parameters:
| Parameter         | Description                                              |
|-------------------|----------------------------------------------------------|
| `unbinned_data`   | Path to filtered GRB FITS file                           |
| `output_name`     | Output HDF5 file name prefix                             |
| `psichi_binning`  | Sky binning system (`"local"` = spacecraft-centric)      |



In [ ]:
from cosipy import BinnedData

# Paths
input_yaml = "/data01/grb/inputs.yaml"
unbinned_fits = "/data01/grb/GRB_bn081207680_3months_unbinned_data_filtered_with_SAAcut.fits"
output_name = "/data01/grb/GRB_bn081207680_binned_O3"  # full path output

# Run binning
analysis = BinnedData(input_yaml)

analysis.get_binned_data(
    unbinned_data=unbinned_fits,
    output_name=output_name,  # full path!
    psichi_binning="local"
)

print(f"✅ Binning complete. Output written: {output_name}.hdf5")

## Run GRB Binning

This block uses the `BinnedData` class to bin the GRB photon events based on the parameters in `inputs.yaml`.

- `input_yaml`: Path to the YAML configuration file.
- `unbinned_fits`: Input FITS file containing GRB event data.
- `output_name`: Full path for the output HDF5 binned dataset.
- `psichi_binning="local"`: Performs sky binning in the spacecraft (local) frame.

The result is saved as `GRB_bn081207680_binned_O3.hdf5`.



In [ ]:
from cosipy import BinnedData
import os

analysis = BinnedData("input_bg.yaml")

analysis.get_binned_data(
    unbinned_data="/data01/grb/Total_BG_with_SAAcomponent_3months_unbinned_data_filtered_with_SAAcut.fits",
    output_name="/data01/grb/Total_BG_binned_O3",
    psichi_binning="local",
    event_range=[0, 100000]  #If you want to download the full background then please remove this line
)

print("background binned successfully")
# print("Output saved to:", os.getcwd())


## Run Background Binning

This block bins the background photon events using a separate configuration file (`input_bg.yaml`).

- `unbinned_data`: Path to the SAA-filtered background FITS file.
- `output_name`: Full path for the output HDF5 file.
- `psichi_binning="local"`: Bins data in the spacecraft-centric frame.
- `event_range=[0, 100000]`: Optional limit on number of events for faster processing (remove to use all events).

The result is saved as `Total_BG_binned_O3.hdf5`.
